In [1]:
!pip install langchain-mistralai
!pip install langchain-community
!pip install chromadb
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U langchain-huggingface

In [3]:
import os
from typing import TypedDict, List, Optional, Dict, Any, Annotated
from typing_extensions import TypedDict
import operator
import json

# Import from your available modules
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver  # Or MemorySaver from langgraph.checkpoint.memory
from langchain_mistralai import ChatMistralAI
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from datetime import datetime
import uuid
from google.colab import userdata
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


# Хелперы, чтобы всегда брать именно последнее сообщение пользователя
def get_last_human_message(messages):
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            return msg
    return None

def get_last_human_content(messages) -> str:
    msg = get_last_human_message(messages)
    if msg is None:
        return ""
    # у HumanMessage текст лежит в .content
    return getattr(msg, "content", str(msg)).strip()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pwd

/content


In [ ]:
def initialize_mistral():
    """Инициализация Mistral через API"""
    # Установите ваш API ключ Mistral
    if "MISTRAL_API_KEY" not in os.environ:
        os.environ["MISTRAL_API_KEY"] = "API-ключ"

    MISTRAL_MODEL = "mistral-large-latest"
    MISTRAL_BASE_URL = "https://api.mistral.ai/v1"

    # Создаем LLM
    llm = ChatMistralAI(
        model="mistral-large-latest",
        temperature=0.1,  # Немного творчества для лучших ответов
        max_retries=3
    )
    return llm

llm = initialize_mistral()

In [7]:
# ============ 2. ОПРЕДЕЛЕНИЕ СОСТОЯНИЯ ============

class AgentState(TypedDict):
    """Состояние агента для управления диалогом"""
    messages: Annotated[List[Dict], add_messages]
    user_request: Optional[str]
    service_type: Optional[str]
    university: Optional[str]
    interests: Optional[str]
    filtered_events: Optional[List[Dict]]
    selected_event: Optional[Dict]
    registration_data: Optional[Dict]
    current_step: str

In [8]:
# ============ 3. БАЗА ДАННЫХ МЕРОПРИЯТИЙ ============

class EventDatabase:
    def __init__(self):
        # Используем мультиязычные эмбеддинги
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        )

        # self.events = [
        #     {
        #         "id": 1,
        #         "title": "Хакатон по машинному обучению",
        #         "university": "МГУ",
        #         "description": "Соревнование по разработке ML-моделей для студентов",
        #         "date": "2024-12-15",
        #         "location": "Главный корпус, ауд. 310",
        #         "interest_tags": ["программирование", "искусственный интеллект", "data science"],
        #         "categories": ["технологии", "IT", "ML"]
        #     },
        #     {
        #         "id": 2,
        #         "title": "Лекция по квантовой физике",
        #         "university": "МГУ",
        #         "description": "Введение в квантовые вычисления",
        #         "date": "2024-12-11",
        #         "location": "Физический факультет, ауд. 202",
        #         "interest_tags": ["физика", "наука", "технологии"],
        #         "categories": ["наука", "физика"]
        #     },
        #     {
        #         "id": 3,
        #         "title": "Философские чтения",
        #         "university": "РАНХиГС",
        #         "description": "Познавательное мероприятие по философии",
        #         "date": "2024-12-17",
        #         "location": "Философский факультет, ауд. 102",
        #         "interest_tags": ["когнитивные технологии", "познание"],
        #         "categories": ["философия", "культура"]
        #     }
        # ]

        # Пример данных мероприятий
        with open("/content/drive/MyDrive/Hugging_face/мероприятия_по_университетам.json", 'r', encoding="utf-8") as f:
          self.events = json.load(f)
          for i in range(len(self.events)):
            self.events[i]['id'] = i+1
            self.events[i]['title'] = self.events[i].pop('Наименование студенческого мероприятия')
            self.events[i]['university'] = self.events[i].pop('наименование университета, в котором проводится мероприятие')
            self.events[i]['description'] = self.events[i].pop('Краткое описание')
            self.events[i]['date'] = self.events[i].pop('дата начала')
            self.events[i]['location'] = self.events[i]['university']
            self.events[i]['interest_tags'] = self.events[i].pop('тематика').split(', ')
            self.events[i]['categories'] = self.events[i]['interest_tags']

            # только удаляем поля
            self.events[i].pop('дата окончания')
            self.events[i].pop('Электронная почта организатора')

          print(json.dumps(self.events, ensure_ascii=False, indent=4))


        # Создаем векторное хранилище
        self.vectorstore = self._create_vector_store()

    def _create_vector_store(self):
        """Создание векторной базы данных"""
        texts = []
        metadatas = []

        for event in self.events:
            # Клонируем метаданные, чтобы не изменять оригинальные данные
            metadata_for_db = event.copy()

            # Преобразуем списки в строки
            if 'interest_tags' in metadata_for_db:
                # Объединяем теги в одну строку через запятую
                metadata_for_db['interest_tags'] = ', '.join(metadata_for_db['interest_tags'])

            if 'categories' in metadata_for_db:
                metadata_for_db['categories'] = ', '.join(metadata_for_db['categories'])

            # Создаем текст для эмбеддинга (оставляем как есть)
            text = f"{event['title']}. {event['description']}. University: {event['university']}. Tags: {', '.join(event['interest_tags'])}"
            texts.append(text)
            metadatas.append(metadata_for_db)  # Используем преобразованные метаданные

        # Создаем Chroma коллекцию
        vectorstore = Chroma.from_texts(
            texts=texts,
            embedding=self.embeddings,
            metadatas=metadatas,  # Теперь здесь только простые типы данных
            collection_name="university_events"
        )
        return vectorstore

    def search_events(self, university: str, interests: str, k: int = 5) -> List[dict]:
        """Поиск мероприятий по университету и интересам"""
        query = f"Events at {university} university about {interests}"

        # Поиск похожих мероприятий
        docs = self.vectorstore.similarity_search(query, k=k)

        # Фильтруем по университету
        # filtered_events = []
        # for doc in docs:
        #     if doc.metadata.get('university', '').lower() == university.lower():
        #         filtered_events.append(doc.metadata)

        # return filtered_events

        filtered_events = []
        univ_user = university.lower()

        for doc in docs:
            univ_meta = doc.metadata.get('university', '').lower()

            # если пользователь ввёл "сфу" → оно содержится в "сибирский федеральный университет (сфу)"
            if univ_user in univ_meta or univ_meta in univ_user:
                filtered_events.append(doc.metadata)

        return filtered_events

    def register_for_event(self, event_id: int, user_data: dict) -> bool:
        """Регистрация на мероприятие"""
        print(f"[Database] Registration for event {event_id}: {user_data}")
        # Здесь можно добавить сохранение в реальную БД
        return True

# Создаем экземпляр базы данных
event_db = EventDatabase()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[
    {
        "id": 1,
        "title": "Зимний хакатон «Цифровой прорыв. Сибирь»",
        "university": "Сибирский федеральный университет (СФУ)",
        "description": "48-часовой марафон по разработке IT-решений для городской среды",
        "date": "2025-12-15",
        "location": "Сибирский федеральный университет (СФУ)",
        "interest_tags": [
            "IT",
            "программирование"
        ],
        "categories": [
            "IT",
            "программирование"
        ]
    },
    {
        "id": 2,
        "title": "Зимняя научная конференция «Неделя науки СФУ»",
        "university": "Сибирский федеральный университет (СФУ)",
        "description": "Конференция с участием студентов, аспирантов и молодых ученых по всем направлениям науки",
        "date": "2026-01-12",
        "location": "Сибирский федеральный университет (СФУ)",
        "interest_tags": [
            "Наука",
            "исследования"
        ],
        "categories": [
            "Наук

In [9]:
# from sentence_transformers import SentenceTransformer
# import numpy as np

# class EventSearch:
#     def __init__(self, events):
#         self.events = events

#         # 1. Уникальные вузы из данных
#         self.universities = sorted({e["university"] for e in self.events})

#         # 2. Модель эмбеддингов
#         self.embed_model = SentenceTransformer(
#             "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
#         )

#         # 3. Предсчитываем эмбеддинги вузов
#         self.univ_embs = self.embed_model.encode(
#             self.universities, normalize_embeddings=True
#         )

#     def resolve_university(self, user_text, threshold=0.6):
#         """Возвращает нормализованное название вуза или None."""
#         q_emb = self.embed_model.encode(
#             [user_text], normalize_embeddings=True
#         )[0]

#         # косинусное сходство = dot product, т.к. нормализовано
#         sims = np.dot(self.univ_embs, q_emb)
#         best_idx = int(np.argmax(sims))
#         best_sim = float(sims[best_idx])

#         if best_sim >= threshold:
#             return self.universities[best_idx], best_sim
#         else:
#             return None, best_sim

#     def find_events(self, user_university_text, user_interest_text):
#         # 1) нормализуем название вуза
#         resolved_univ, sim = self.resolve_university(user_university_text)
#         if resolved_univ is None:
#             return [], f"Не смогли сопоставить вуз (max similarity={sim:.2f})"

#         # 2) фильтруем мероприятия по вузу (строго)
#         events_for_univ = [
#             e for e in self.events if e["university"] == resolved_univ
#         ]

#         # 3) дальше твоя логика по interest_tags (пока по строковому совпадению)
#         interest = user_interest_text.strip().lower()
#         matched = []
#         for e in events_for_univ:
#             tags = [t.lower() for t in e["interest_tags"]]
#             if interest in tags:
#                 matched.append(e)

#         return matched, resolved_univ

In [10]:
# ============ 4. ОПРЕДЕЛЕНИЕ УЗЛОВ ГРАФА ============

def classify_service(state: AgentState) -> AgentState:
    """
    Единый узел-контроллер:
    - на первом шаге классифицирует запрос (1–4),
    - далее по current_step решает, какой следующий шаг выполнять.
    """
    messages = state["messages"]
    step = state.get("current_step")
    service_type = state.get("service_type")

    # ---------- 1. Если это НЕ первый шаг диалога — роутим по current_step ----------

    # Пользователь только что прислал название вуза
    if step == "awaiting_university":
        return process_university(state)

    # Пользователь только что прислал свои интересы
    if step == "awaiting_interests":
        return search_events(state)

    # Пользователь сейчас отвечает "да/нет" на вопрос о записи
    if step == "events_presented":
        return handle_registration_decision(state)

    # Пользователь выбирает номер мероприятия
    if step == "awaiting_event_selection":
        return process_event_selection(state)

    # Пользователь присылает ФИО и номер студенческого
    if step == "awaiting_registration_data":
        return complete_registration(state)

    # Диалог уже в терминальном состоянии — просто вежливо отвечаем
    if step in ("no_events_found", "end_conversation", "dialog_finished"):
        goodbye = "Если появятся ещё вопросы по мероприятиям в вузе — напишите 🙂"
        return {
            **state,
            "messages": messages + [AIMessage(content=goodbye)],
            "current_step": "dialog_finished",
        }

    # ---------- 2. Первый шаг: нужно классифицировать запрос ----------

    # Берём последнее пользовательское сообщение
    last_message = get_last_human_message(messages)
    if last_message is None:
        user_content = ""
    elif hasattr(last_message, "content"):
        user_content = last_message.content
    else:
        user_content = str(last_message)

    # Создаём клиент Mistral
    mistral_api_key = os.environ.get("MISTRAL_API_KEY", "<ВАШ_MISTRAL_API_KEY>")

    llm_client = ChatMistralAI(
        model="mistral-small-latest",
        temperature=0,
        max_retries=2,
        mistral_api_key=mistral_api_key,
    )

    system_prompt = """Ты классифицируешь запросы студентов. Определи номер услуги:
    1) Штрафы на машину
    2) Льготы
    3) Подбор университетов
    4) Мероприятия в вузе

    Ответь ТОЛЬКО цифрой (1, 2, 3 или 4)."""

    llm_messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_content),
    ]

    try:
        response = llm_client.invoke(llm_messages)
        service_type = response.content.strip()

        if service_type not in ["1", "2", "3", "4"]:
            print(f"[WARN] LLM вернул неожиданный ответ: '{service_type}'")
            # Fallback — если явно есть слова про мероприятия, считаем, что это 4-я услуга
            if any(word in user_content.lower() for word in ["мероприятия", "ивент", "события", "активности"]):
                service_type = "4"
            else:
                service_type = "1"
    except Exception as e:
        print(f"[ERROR] Ошибка при вызове Mistral API: {e}")
        if any(word in user_content.lower() for word in ["мероприятия", "ивент", "события", "активности"]):
            service_type = "4"
        else:
            service_type = "1"

    # Обновляем базовое состояние
    base_state: AgentState = {
        **state,
        "service_type": service_type,
    }

    # ---------- 3. Обрабатываем только ветку 4 (мероприятия) ----------
    if service_type == "4":
        # Сразу переходим к вопросу об университете
        return ask_university(base_state)

    # Остальные ветки сейчас не реализованы — честно говорим об этом
    response_text = (
        "Сейчас я умею помогать только с подбором мероприятий в вузе. "
        "Попробуйте сформулировать запрос про мероприятия 🙂"
    )

    return {
        **base_state,
        "messages": messages + [AIMessage(content=response_text)],
        "current_step": "unsupported_service",
    }


def process_university(state: AgentState) -> AgentState:
    """Обработка ответа с университетом"""
    messages = state['messages']

    # ❗ Берём именно последнее сообщение пользователя
    university = get_last_human_content(messages)

    # Спрашиваем об интересах
    response = (
        "Спасибо! Теперь расскажите, какие направления вас интересуют? "
        "Например: программирование, дизайн, наука, спорт, искусство и т.д."
    )

    return {
        **state,
        "university": university,
        "messages": messages + [AIMessage(content=response)],
        "current_step": "awaiting_interests",
    }

def ask_university(state: AgentState) -> AgentState:
    messages = state['messages']
    question = "Для подбора мероприятий, пожалуйста, укажите название вашего университета."
    return {
        **state,
        "messages": messages + [AIMessage(content=question)],
        "current_step": "awaiting_university"
    }

def search_events(state: AgentState) -> AgentState:
    """Поиск мероприятий в базе данных"""
    messages = state['messages']

    # ❗ Берём интересы именно из последнего HumanMessage
    interests = get_last_human_content(messages)

    # Ищем мероприятия
    events = event_db.search_events(
        university=state['university'],
        interests=interests
    )

    # Если мероприятий нет
    if not events:
        response = f"К сожалению, для университета {state['university']} и интересов '{interests}' мероприятий не найдено."
        return {
            **state,
            "interests": interests,
            "filtered_events": [],
            "messages": messages + [AIMessage(content=response)],
            "current_step": "no_events_found"
        }

    # Форматируем список мероприятий
    events_list = "\n".join([
        f"{i+1}. {event['title']}\n   📅 {event['date']}\n   📍 {event['location']}\n   {event['description']}\n"
        for i, event in enumerate(events)
    ])

    response = f"Нашёл для вас мероприятия в {state['university']}:\n\n{events_list}\n\nХотите записаться на какое-либо из этих мероприятий? (да/нет)"

    return {
        **state,
        "interests": interests,
        "filtered_events": events,
        "messages": messages + [AIMessage(content=response)],
        "current_step": "events_presented"
    }

def handle_registration_decision(state: AgentState) -> AgentState:
    """Обработка решения о регистрации"""
    messages = state['messages']

    # ❗ Берём последний ответ пользователя
    user_response = get_last_human_content(messages).lower()

    # Анализируем ответ пользователя
    if any(word in user_response for word in ["да", "хочу", "конечно", "запишите"]):
        response = "Отлично! На какое мероприятие вы хотите записаться? Укажите номер из списка."
        next_step = "awaiting_event_selection"
    else:
        response = "Хорошо! Если понадобится помощь с мероприятиями в будущем, обращайтесь. Всего доброго!"
        next_step = "end_conversation"

    return {
        **state,
        "messages": messages + [AIMessage(content=response)],
        "current_step": next_step
    }

def process_event_selection(state: AgentState) -> AgentState:
    """Обработка выбора мероприятия"""
    messages = state['messages']

    # ❗ Берём номер мероприятия из последнего HumanMessage
    user_input = get_last_human_content(messages)

    try:
        event_index = int(user_input) - 1
        events = state['filtered_events']

        # Проверяем корректность выбора
        if 0 <= event_index < len(events):
            selected_event = events[event_index]
            response = f"Вы выбрали: {selected_event['title']}\n\nДля регистрации, пожалуйста, предоставьте:\n1. Ваше ФИО\n2. Номер студенческого билета\n\nВведите данные в формате: ФИО, номер билета"

            return {
                **state,
                "selected_event": selected_event,
                "messages": messages + [AIMessage(content=response)],
                "current_step": "awaiting_registration_data"
            }
    except ValueError:
        # Неверный формат номера
        pass

    # Если что-то пошло не так
    response = "Пожалуйста, укажите корректный номер мероприятия из списка."
    return {
        **state,
        "messages": messages + [AIMessage(content=response)],
        "current_step": "awaiting_event_selection"
    }

def complete_registration(state: AgentState) -> AgentState:
    """Завершение регистрации"""
    messages = state['messages']

    # ❗ Берём строку с ФИО и студ билетом из последнего HumanMessage
    user_input = get_last_human_content(messages)

    try:
        # Парсим данные пользователя
        parts = [part.strip() for part in user_input.split(',', 1)]

        if len(parts) >= 2:
            fio = parts[0]
            student_id = parts[1]

            registration_data = {
                "fio": fio,
                "student_id": student_id,
                "registration_date": datetime.now().isoformat()
            }

            # Регистрируем пользователя
            success = event_db.register_for_event(
                state['selected_event']['id'],
                registration_data
            )

            if success:
                response = f"""
✅ Регистрация успешно завершена!

📋 Детали мероприятия:
Название: {state['selected_event']['title']}
Дата: {state['selected_event']['date']}
Место: {state['selected_event']['location']}

📝 Ваши данные:
ФИО: {fio}
Номер студенческого: {student_id}

Ждём вас на мероприятии! Всего доброго!"""
            else:
                response = "Произошла ошибка при регистрации. Пожалуйста, попробуйте позже."
        else:
            response = "Пожалуйста, введите данные в правильном формате: ФИО, номер студенческого билета"
            return {
                **state,
                "messages": messages + [AIMessage(content=response)],
                "current_step": "awaiting_registration_data"
            }

    except Exception as e:
        response = f"Произошла ошибка при обработке данных: {str(e)}"
        return {
            **state,
            "messages": messages + [AIMessage(content=response)],
            "current_step": "awaiting_registration_data"
        }

    return {
        **state,
        "registration_data": registration_data,
        "messages": messages + [AIMessage(content=response)],
        "current_step": "registration_completed"
    }

In [11]:
def create_agent_graph():
    """Создание workflow графа с одним управляющим узлом."""

    workflow = StateGraph(AgentState)

    # Один узел — наш контроллер/агент
    workflow.add_node("agent", classify_service)

    # Точка входа — этот же узел
    workflow.set_entry_point("agent")

    # Память по thread_id
    memory = MemorySaver()

    app = workflow.compile(checkpointer=memory)
    return app

In [12]:
# ============ 6. ФУНКЦИЯ ДЛЯ ОБЩЕНИЯ ============

def chat_with_agent(user_input: str, thread_id: str = "default"):
    """Основная функция для общения с агентом"""

    # Создаем граф
    agent = create_agent_graph()

    # Конфигурация для сохранения состояния
    config = {"configurable": {"thread_id": thread_id}}

    # Используем правильный формат сообщений
    input_messages = [HumanMessage(content=user_input)]

    # Запускаем обработку сообщения
    try:
        events = agent.stream(
            {"messages": input_messages},
            config,
            stream_mode="values"
        )

        # Обрабатываем результаты
        last_state = None
        for event in events:
            if 'messages' in event and event['messages']:
                last_message = event['messages'][-1]
                if isinstance(last_message, AIMessage):
                    print(f"🤖 Ассистент: {last_message.content}")
                elif hasattr(last_message, 'role') and last_message.role == "assistant":
                    # Для совместимости со старым форматом
                    print(f"🤖 Ассистент: {last_message.content}")
            last_state = event

        return last_state

    except Exception as e:
        print(f"❌ Ошибка: {e}")
        return None


# ============ 7. ПРИМЕР ИСПОЛЬЗОВАНИЯ ============

def run_example_conversation():
    """Пример полного диалога с агентом"""

    print("=" * 50)
    print("👋 Добро пожаловать! Чем могу помочь?")
    print("=" * 50)

    # Создаем граф
    agent = create_agent_graph()
    thread_id = "example_conversation"
    config = {"configurable": {"thread_id": thread_id}}

    # Симуляция диалога
    conversation_steps = [
        ("Хочу найти мероприятия в моём университете", "1. Инициируем запрос"),
        ("МГУ", "2. Указываем университет"),
        ("программирование и искусственный интеллект", "3. Указываем интересы"),
        ("да", "4. Решаем о регистрации"),
        ("1", "5. Выбираем мероприятие"),
        ("Иванов Иван Иванович, СТУ12345", "6. Предоставляем данные")
    ]

    # Начинаем диалог
    current_messages = []

    for user_input, step_description in conversation_steps:
        print(f"\n{step_description}:")
        print(f"👤 Пользователь: {user_input}")

        # Добавляем сообщение пользователя
        current_messages.append(HumanMessage(content=user_input))

        try:
            # Запускаем обработку
            events = agent.stream(
                {"messages": current_messages},
                config,
                stream_mode="values"
            )

            # Получаем ответ агента
            for event in events:
                if 'messages' in event and event['messages']:
                    # Обновляем текущие сообщения
                    current_messages = event['messages']

                    # Выводим последний ответ ассистента
                    last_message = current_messages[-1]
                    if isinstance(last_message, AIMessage):
                        print(f"🤖 Ассистент: {last_message.content}")
                    elif hasattr(last_message, 'content'):
                        print(f"🤖 Ассистент: {last_message.content}")

        except Exception as e:
            print(f"❌ Ошибка на шаге '{step_description}': {e}")
            break

    print("\n" + "=" * 50)
    print("✅ Диалог завершен!")
    print("=" * 50)


In [13]:
run_example_conversation()

👋 Добро пожаловать! Чем могу помочь?

1. Инициируем запрос:
👤 Пользователь: Хочу найти мероприятия в моём университете
🤖 Ассистент: Хочу найти мероприятия в моём университете
🤖 Ассистент: Для подбора мероприятий, пожалуйста, укажите название вашего университета.

2. Указываем университет:
👤 Пользователь: МГУ
🤖 Ассистент: МГУ
🤖 Ассистент: Спасибо! Теперь расскажите, какие направления вас интересуют? Например: программирование, дизайн, наука, спорт, искусство и т.д.

3. Указываем интересы:
👤 Пользователь: программирование и искусственный интеллект
🤖 Ассистент: программирование и искусственный интеллект
🤖 Ассистент: К сожалению, для университета МГУ и интересов 'программирование и искусственный интеллект' мероприятий не найдено.

4. Решаем о регистрации:
👤 Пользователь: да
🤖 Ассистент: да
🤖 Ассистент: Если появятся ещё вопросы по мероприятиям в вузе — напишите 🙂

5. Выбираем мероприятие:
👤 Пользователь: 1
🤖 Ассистент: 1
🤖 Ассистент: Если появятся ещё вопросы по мероприятиям в вузе — напиш

In [14]:
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import HumanMessage
import os

# Убедитесь, что ключ задан
os.environ["MISTRAL_API_KEY"] = "iWsZEMRGjFiiL3yO53peiw4rVuURZMgA"

llm = ChatMistralAI(model="mistral-small-latest", temperature=0)
messages = [
    {"role": "user", "content": "Привет, это тест."}
    # ИЛИ попробуйте формат с HumanMessage
    # HumanMessage(content="Привет, это тест.")
]

try:
    response = llm.invoke(messages)
    print(f"✅ Ответ получен: {response.content}")
except Exception as e:
    print(f"❌ Ошибка при прямом вызове: {e}")

✅ Ответ получен: Привет! 😊 Да, это тест. Если у тебя есть вопросы или что-то нужно проверить — спрашивай! Готов помочь. 🚀


In [15]:
def interactive_chat():
    """Интерактивный чат с агентом"""
    print("=" * 60)
    print("🤖 АГЕНТ ДЛЯ ПОИСКА МЕРОПРИЯТИЙ В ВУЗАХ")
    print("=" * 60)
    print("Я помогу найти мероприятия в вашем университете.")
    print("Команды: 'выход' - завершить диалог, 'сброс' - начать заново")
    print("-" * 60)

    # Создаем агента
    agent = create_agent_graph()
    thread_id = f"chat_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    config = {"configurable": {"thread_id": thread_id}}

    # История сообщений
    messages = []

    while True:
        try:
            # Запрос ввода пользователя
            user_input = input("\n👤 Вы: ").strip()

            # Проверка команд
            if user_input.lower() in ['выход', 'exit', 'quit', 'q']:
                print("\n🤖 Ассистент: До свидания! Возвращайтесь, когда понадобится помощь с мероприятиями.")
                break

            if user_input.lower() in ['сброс', 'reset', 'новый', 'new']:
                messages = []
                thread_id = f"chat_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                config = {"configurable": {"thread_id": thread_id}}
                print("\n🤖 Ассистент: Диалог сброшен. Начнем заново!")
                continue

            # Добавляем сообщение пользователя
            messages.append(HumanMessage(content=user_input))

            # Вызываем агента
            result = agent.invoke({"messages": messages}, config)

            # Обновляем историю
            messages = result['messages']

            # Выводим последний ответ ассистента
            last_message = messages[-1]
            if isinstance(last_message, AIMessage):
                print(f"🤖 Ассистент: {last_message.content}")
            else:
                # Если последнее сообщение не от ассистента, ищем последнее AIMessage
                for msg in reversed(messages):
                    if isinstance(msg, AIMessage):
                        print(f"🤖 Ассистент: {msg.content}")
                        break

            # Отладочная информация (опционально)
            if 'current_step' in result:
                print(f"[Шаг: {result['current_step']}]")

        except KeyboardInterrupt:
            print("\n\n🤖 Ассистент: Диалог прерван. До свидания!")
            break
        except Exception as e:
            print(f"\n❌ Ошибка: {e}")
            print("Попробуйте снова или введите 'сброс' для начала нового диалога.")

In [18]:
interactive_chat()

🤖 АГЕНТ ДЛЯ ПОИСКА МЕРОПРИЯТИЙ В ВУЗАХ
Я помогу найти мероприятия в вашем университете.
Команды: 'выход' - завершить диалог, 'сброс' - начать заново
------------------------------------------------------------

👤 Вы: Я хочу записаться на мероприятие
🤖 Ассистент: Для подбора мероприятий, пожалуйста, укажите название вашего университета.
[Шаг: awaiting_university]

👤 Вы: СФУ
🤖 Ассистент: Спасибо! Теперь расскажите, какие направления вас интересуют? Например: программирование, дизайн, наука, спорт, искусство и т.д.
[Шаг: awaiting_interests]

👤 Вы: наука
🤖 Ассистент: Нашёл для вас мероприятия в СФУ:

1. Зимний научный стендап «Science Slam СФУ»
   📅 2026-01-28
   📍 Сибирский федеральный университет (СФУ)
   Молодые ученые в неформальной обстановке рассказывают о своих исследованиях

2. Зимняя научная конференция «Неделя науки СФУ»
   📅 2026-01-12
   📍 Сибирский федеральный университет (СФУ)
   Конференция с участием студентов, аспирантов и молодых ученых по всем направлениям науки

3. Конк

In [17]:
# мероприятие
# СФУ
# программирование

#Я учусь в МГУ, интересуюсь квантовыми вычислениями и хочу узнать о мероприятиях в моём вузе
# машинное обучение

In [ ]:
СибГУ